# Blockhouse Task – Excel-based Slippage Computation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-notebook')

In [ ]:
def compute_slippage_row(row, x):
    ask_prices = [row[f'ask_px_0{i}'] for i in range(10)]
    ask_sizes  = [row[f'ask_sz_0{i}'] for i in range(10)]
    bid_price = row['bid_px_00']
    ask_price = row['ask_px_00']

    cumulative_size = 0
    cost = 0
    for px, sz in zip(ask_prices, ask_sizes):
        if cumulative_size + sz <= x:
            cost += px * sz
            cumulative_size += sz
        else:
            remaining = x - cumulative_size
            cost += px * remaining
            cumulative_size += remaining
            break
    if cumulative_size < x:
        return np.nan
    avg_price = cost / x
    mid_price = (bid_price + ask_price) / 2
    return avg_price - mid_price

In [ ]:
# 📥 Replace the path below with your actual Excel file name
file_path = "/mnt/data/your_file.xlsx"  # <-- Update this after upload
df = pd.read_excel(file_path)

print("Loaded rows:", len(df))

In [ ]:
x = 200  # Change as needed
df['slippage_x_200'] = df.apply(lambda row: compute_slippage_row(row, x), axis=1)
df[['ts_event', 'slippage_x_200']].head()

In [ ]:
df[['ts_event', 'slippage_x_200']].plot(x='ts_event', y='slippage_x_200', figsize=(12, 4), title=f'Slippage Over Time for x = {x}')
plt.xlabel("Timestamp")
plt.ylabel("Slippage ($)")
plt.grid(True)
plt.show()